# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных уже сделана.

Необходимо построить модель с максимально большим значением accuracy. Нужно довести долю правильных ответов по крайней мере до 0.75 и проверить accuracy на тестовой выборке.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [3]:
display(data.head(10))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


## Разбейте данные на выборки

In [6]:
#поделим датафрейм на обучающую выборку и выборку, которую позже разделим на валидационную и тестовую
data_train, data_other = train_test_split(data, test_size=0.4, random_state=12345)
#поделим data_other на валидационную и тестовую выборку
data_valid, data_test = train_test_split(data_other, test_size=0.5, random_state=12345)
display(data_train.shape)
display(data_valid.shape)
display(data_test.shape)

(1928, 5)

(643, 5)

(643, 5)

## Исследуйте модели

In [7]:
#разделим обучающий датафрейм на признаки и целевой признак
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

#разделим валидационный датафрейм на признаки и целевой признак
features_valid = data_valid.drop(['is_ultra'], axis=1)  
target_valid = data_valid['is_ultra']

In [8]:
#исследуем модель дерево решений
best_accuracy = 0
best_depth = 0 
for depth in range(1,16):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева =", best_depth, "Accuracy наилучшей модели на валидационной выборке:", best_accuracy)

Глубина дерева = 3 Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


лучшая модель с глубиной 3 и с `accuracy` = 0.7853

In [9]:
#исследуем модель случайный лес
best_est = 0
best_accuracy_forest = 0
best_depth = 0
for depth in range(1, 16):
    for est in range(3, 58):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        if accuracy > best_accuracy_forest:
            best_est = est
            best_accuracy_forest = accuracy
            best_depth = depth
print("Количество деревьев =", best_est, "Глубина дерева", best_depth, "Accuracy наилучшей модели на валидационной выборке:", best_accuracy_forest)

Количество деревьев = 40 Глубина дерева 8 Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247


лучшая модель с количеством деревьев 40, глубиной дерева 8 и с `accuracy` = 0.808

In [10]:
#исследуем модель логистической регрессии
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print("Accuracy наилучшей модели на валидационной выборке:", accuracy)

Accuracy наилучшей модели на валидационной выборке: 0.7107309486780715


`accuracy` = 0.7107, результат хуже чем у дерева решений и модели случайного леса

`Вывод:` 
- Обучили дерево принятия решений и выбрали лучшее с максимальной глубиной 3 и `accuracy` = 0.7853
- Обучили модель случайного леса и выбрали лучшую с количеством деревьев 40, глубиной 8 и `accuracy` = 0.808
- Обучили модель логистической регресии и получили `accuracy` =  0.7107, результат хуже чем у других моделей

Лучший результат показала модель случайного леса, ее будем использовать для проверки на тестовой выборке

## Проверьте модель на тестовой выборке

In [11]:
#разделим тестовый датафрейм на признаки и целевой признак
features_test = data_test.drop(['is_ultra'], axis=1)  
target_test = data_test['is_ultra']

In [12]:
#проверим модель случайного леса на тестовой выборке
features_general = pd.concat([features_valid, features_train], ignore_index= True)
target_general = pd.concat([target_valid, target_train], ignore_index= True)
model = RandomForestClassifier(random_state=12345, n_estimators=40, max_depth=8)
model.fit(features_general, target_general)
predictions = model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
print('accuracy =', accuracy)

accuracy = 0.7993779160186625


Модель показала хороший результат на тестовой выборке

## Проверьте модели на адекватность

In [13]:
#проверим модель на адекватность
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy_model.fit(features_train, target_train)
print('accuracy =', dummy_model.score(features_test, target_test))

accuracy = 0.6842923794712286


`accuracy` на тестовой выборке больше чем при проверке классификатором `DummyClassifier`, значит наша модель прошла проверка на адекватность

# Общий вывод:

1. Были изучены и скорректированы данные.

2. Сделана разбивка на три выборки: обучающая, валидация и тестовая.

3. Были рассмотрены три модели классификации (дерево решений, случайный лес и логическая регрессия), изменены гиперпараметры.

4. Было выявлено, что наилучшей валидационной и тестовой моделью является случайный лес (80%), что выше, чем распределение тарифов в данных, то есть тестовая модель не предсказывает 0 или 1.